In [ ]:
""" Описание столбцов:
Index — номер строки
Suburb — наименование пригорода
Address — адрес
Rooms — количество комнат в помещении
Type — тип здания (h — дом, коттедж, вилла, терраса; u — блочный, дуплексный дом; t — таунхаус)
Price — цена помещения
Method — метод продажи 
SellerG — риэлторская компания
Date — дата продажи (в формате день/месяц/год)
Distance — расстояния до объекта от центра Мельбурна 
Postcode — почтовый индекс
Bedroom — количество спален
Bathroom — количество ванных комнат
Car — количество парковочных мест
Landsize — площадь прилегающей территории
BuildingArea — площадь здания
YearBuilt — год постройки
CouncilArea — региональное управление
Lattitude — географическая широта
Longitude — географическая долгота
Regionname — наименование района Мельбурна
Propertycount — количество объектов недвижимости в районе
Coordinates — широта и долгота, объединённые в кортеж
"""


In [35]:
# базовые операции со столбцами

import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()

melb_df = melb_df.drop(['index', 'Coordinates'], axis=1) # порядковые номера или имена столбцов для удаления, если их несколько, передаётся список
# axis=0 — удаляются строки, axis=1 — удаляются столбцы

total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom'] # общее количество комнат
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms # средняя площадь одной комнаты

diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area # соотношение площади здания и участка
# <0 площадь участка больше площади здания, -1 без строений, 1 без территорий

In [36]:
# дата и время

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True) # приведение к типу datetime64, dayfirst=True - первым изначально шел день

years_sold = melb_df['Date'].dt.year
# date, year, month, day, time, hour, minute, second, dayofweek, day_name, dayofyear, quarter
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)*100

delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') # разница в днях, Series c типом данных timedelta64
delta_days.dt.days # приведение к формату int64

melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt'] # возраста объекта недвижимости в годах на момент продажи
melb_df = melb_df.drop('YearBuilt', axis=1) # удаляем дублирующий столбец YearBuilt
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek

def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else: 
        return 0
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)

Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


In [37]:
# создание и преобразование столбцов

def get_street_type(address): # вытаскиваем подтип улицы из адреса
    exclude_list = ['N', 'S', 'W', 'E']
    address_list = address.split(' ')
    street_type = address_list[-1]
    if street_type in exclude_list:
        street_type = address_list[-2]
    return street_type

street_types = melb_df['Address'].apply(get_street_type) # применение функции к каждому элементу столбца
popular_stypes = street_types.value_counts().nlargest(10).index # nlargest(n) - возвращает n наибольших значений
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other') 
melb_df = melb_df.drop('Address', axis=1)

pop_sellers = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in pop_sellers else 'other')

In [31]:
# задание: опыт работы из hh

import pandas as pd

def get_experience(arg):
    month_key_words = ['месяц', 'месяцев', 'месяца']
    year_key_words = ['год', 'лет', 'года']
    args_splited = arg.split(' ')
    month = 0
    year = 0
    for i in range(len(args_splited)):
        if args_splited[i] in month_key_words:
            month = args_splited[i-1]
        if args_splited[i] in year_key_words:
            year = args_splited[i-1]
    return int(year)*12 + int(month)



experience_col = pd.Series([
    'Опыт работы 8 лет 3 месяца',
    'Опыт работы 3 года 5 месяцев',
    'Опыт работы 1 год 9 месяцев',
    'Опыт работы 3 месяца',
    'Опыт работы 6 лет'
    ])
experience_month = experience_col.apply(get_experience)
print(experience_month)

0    99
1    -1
2    -1
3     3
4    72
dtype: int64


In [57]:
# типа данных Category

unique_list = []
for col in melb_df.columns:
    item = [col, melb_df[col].nunique(),melb_df[col].dtype]
    unique_list.append(item) 
unique_counts = pd.DataFrame( # таблица с уникальными значениями в каждом столбце
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True) # сортировка DataFrame 

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: 
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: 
        melb_df[col] = melb_df[col].astype('category') # преобразование типа столбца в Category
melb_df['Regionname'].cat.categories # получение списка со значениями
melb_df['Regionname'].cat.codes # каким образом столбец кодируется в виде чисел

melb_df['Type'] = melb_df['Type'].cat.rename_categories({ # переименование категорий с помощью словаря
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
    })

# тип данных category хранит только категории, которые были объявлены при его инициализации
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat') # добавление нового типа категорий, текущие категории не меняются

suburb_types = melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in suburb_types else 'other') 
melb_df['Suburb'] = melb_df['Suburb'].astype('category')

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

In [82]:
# задание: велоспорт

import pandas as pd

citibike_data = pd.read_csv('data/citibike-tripdata.csv', sep=',') 


0    33887
Name: bikeid, dtype: int64

In [100]:
# задание: велоспорт

import pandas as pd

citibike_data = pd.read_csv('data/citibike-tripdata.csv', sep=',')
display(citibike_data)

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,2018-09-01 00:00:20.6490,2018-09-01 00:55:58.5470,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,2018-09-01 00:00:21.7460,2018-09-01 00:07:38.5830,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,2018-09-01 00:00:27.3150,2018-09-01 02:21:25.3080,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,2018-09-05 19:08:27.8460,2018-09-05 19:15:51.4940,494.0,W 26 St & 8 Ave,40.747348,-73.997236,448.0,W 37 St & 10 Ave,40.756604,-73.997901,19531,Subscriber,1990,2
299996,2018-09-05 19:08:28.3700,2018-09-05 19:20:01.5080,3016.0,Kent Ave & N 7 St,40.720368,-73.961651,3086.0,Graham Ave & Conselyea St,40.715143,-73.944507,34020,Subscriber,1982,1
299997,2018-09-05 19:08:27.5090,2018-09-05 19:13:40.5060,3686.0,Gansevoort St & Hudson St,40.739448,-74.005070,168.0,W 18 St & 6 Ave,40.739713,-73.994564,33220,Subscriber,1991,1
299998,2018-09-05 19:08:29.2300,2018-09-05 20:04:29.3220,254.0,W 11 St & 6 Ave,40.735324,-73.998004,499.0,Broadway & W 60 St,40.769155,-73.981918,34744,Subscriber,1975,0


In [101]:
citibike_data['end station name'].value_counts()

West St & Chambers St          1963
12 Ave & W 40 St               1909
Pershing Square North          1850
Central Park S & 6 Ave         1828
E 17 St & Broadway             1804
                               ... 
Exchange Place                    3
Union St                          1
Warren St                         1
Montrose Ave & Bushwick Ave       1
Liberty Light Rail                1
Name: end station name, Length: 765, dtype: int64